In [17]:
import sys
sys.path.append("..")
import Essentials

In [18]:
import numpy as np
from sklearn.preprocessing import StandardScaler
import gmdh
from gmdh import Mia, split_data
from sklearn.model_selection import RandomizedSearchCV

In [19]:
a=np.load('Data_Average.npz', allow_pickle=True)

# Загрузка и подготовка данных

x1=a['X_test_1']
x2=a['X_train_1']

y1=a['Y_test_1']
y2=a['Y_train_1']

timestamp1=y1[:, 1]
timestamp2=y2[:, 1]

y1 = y1[:, 0].reshape(len(y1), 1)
y1 = y1.astype(np.float64)
y2 = y2[:, 0].reshape(len(y2), 1)
y2 = y2.astype(np.float64)

C:\Users\PC\AppData\Local\Temp\ipykernel_14120\4214278781.py:15: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y1 = y1.astype(np.float64)
C:\Users\PC\AppData\Local\Temp\ipykernel_14120\4214278781.py:15: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y1 = y1.astype(np.float64)
C:\Users\PC\AppData\Local\Temp\ipykernel_14120\4214278781.py:15: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y1 = y1.astype(np.float64)
C:\Users\PC\AppData\Local\Temp\ipykernel_14120\42142787

In [20]:
class gmdh_Mia(Essentials.SoftSensor):
    def __init__(self, x_train, y_train):
        super().__init__('Test')
        self.model = gmdh.Mia()
        self.x_scaler = StandardScaler()
        self.y_scaler = StandardScaler()
        self.x_scaler.fit(x_train)
        self.y_scaler.fit(y_train)
        self.RandomizedSearchCV(x_train, y_train)
        self.train(x_train, y_train)

    def prepocessing(self, x):
        try:
            return self.x_scaler.transform(x)
        except:
            try:
                return self.y_scaler.transform(x)
            except BaseException as err:
                print("Ошибка скейлера")
                raise err

    def postprocessing(self, x):
        try:
            return self.x_scaler.inverse_transform(x)
        except:
            try:
                return self.y_scaler.inverse_transform(x)
            except BaseException as err:
                print("Ошибка скейлера")
                raise err
    def RandomizedSearchCV(self,x_train,y_train):
            x_train_scaled = self.prepocessing(x_train) 
            y_train_scaled = self.prepocessing(y_train) 
            param_distributions = {
                'k_best': np.arange(1,12, 1),
                'test_size': [0.2,0.5,0.05],
                'limit': [0.01, 0.1, 0.2],
                'p_average': np.arange(1, 5, 1),
            }
    
            self.random_search = RandomizedSearchCV(
                estimator=self.model,
                param_distributions=param_distributions,
                n_iter=50,
                scoring='neg_mean_squared_error',
                cv=3, 
                verbose=1,
                random_state=42,
                n_jobs=-1  
            )
            self.random_search.fit(x_train_scaled, y_train_scaled)
    
    def evaluate_model(self, x):
        predictions = self.get_model().predict(x)
        return predictions.reshape(-1, 1)

    def train(self, x_train, y_train):
        preproc_y = self.prepocessing(y_train)
        preproc_x = self.prepocessing(x_train)
        self.model.fit(preproc_x, preproc_y, k_best=6,test_size=0.3)
        self.set_model(self.model)

    def __str__(self):
        model=self.get_model()
        return f"Наилучшая найденная модель: \n = {model.get_best_polynomial()}"

In [21]:
Test_sensor_1=gmdh_Mia(x2, y2)

TypeError: Cannot clone object '<gmdh.gmdh.Mia object at 0x00000257B8F1F290>' (type <class 'gmdh.gmdh.Mia'>): it does not seem to be a scikit-learn estimator as it does not implement a 'get_params' method.

In [ ]:
metric = Essentials.R2Metric()
Test_sensor_1.test(x1, y1, metric)

In [ ]:
test_visual=Essentials.Visualizer(x1, y1, timestamp1, [metric], 'Test gmdh Mia Sensor R2 metric')
test_visual.visualize([Test_sensor_1])